In [2]:
import os
import numpy as np
from scipy import stats
import nibabel as nib

from scipy.spatial.distance import dice
from skimage.metrics import hausdorff_distance
from sklearn.metrics import jaccard_score, cohen_kappa_score
from scipy.stats import wilcoxon, ttest_rel

In [3]:
def load_nifti_file(filepath):
    """ Load a NIfTI file and return the image data as a numpy array. """
    img = nib.load(filepath)
    return img.get_fdata()

def calculate_volume(nifti_file):
    """
    Calculate the volume of non-zero values in a NIfTI file.

    Parameters:
    nifti_file (str): The path to the NIfTI file.

    Returns:
    int: The volume of non-zero values in the NIfTI file.
    """
    img = nib.load(nifti_file)
    data = img.get_fdata()
    volume = np.count_nonzero(data)
    return volume

def calculate_dice(nifti_file1, nifti_file2):
    """
    Calculates the Dice coefficient between two NIfTI files.

    Parameters:
    nifti_file1 (str): Path to the first NIfTI file.
    nifti_file2 (str): Path to the second NIfTI file.

    Returns:
    float: The Dice coefficient between the two NIfTI files.
    """
    img1 = nib.load(nifti_file1)
    data1 = img1.get_fdata()

    img2 = nib.load(nifti_file2)
    data2 = img2.get_fdata()

    intersection = np.sum(data1 * data2)
    dice = (2. * intersection) / (np.sum(data1) + np.sum(data2))
    return dice

def perform_stat_test(path, stat_func):
    """
    Perform a statistical test on the T1CM and T1XFLAIR values.

    Args:
        path (str): The path to the directory containing the subject data.
        stat_func (function): The statistical function to be applied to the T1CM and T1XFLAIR values.

    Returns:
        stat_test (tuple): A tuple containing the test statistic and the p-value.

    """
    t1cm_values = []
    t1xflair_values = []

    # List all directories in the given path
    subject_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

    for id in subject_dirs:
        t1cm_file = os.path.join(path, id, f"{id}_t1cm.nii")
        t1xflair_file = os.path.join(path, id, f"{id}_t1xflair.nii")

        if os.path.exists(t1cm_file) and os.path.exists(t1xflair_file):
            value = stat_func(t1cm_file, t1xflair_file)
            t1cm_values.append(value)

    stat_test = stats.ttest_ind(t1cm_values, t1xflair_values)
    return stat_test

- DSC: Values close to 1 (e.g., >0.8): high similarity
- Jaccard Index: Values close to 1 (e.g., >0.7):  high similarity
- Cohen's Kappa: Values > 0.6: substantial agreement; values > 0.8: almost perfect agreement
- Hausdorff Distance: Lower values: high similarity. values close to 0 are desired.
- Volume: Similar volumes indicate similar segmentation extents. 

In [ ]:

def load_nifti_file(filepath):
    """ Load a NIfTI file and return the image data as a numpy array. """
    img = nib.load(filepath)
    return img.get_fdata()

def dice_coefficient(mask1, mask2):
    """ Compute the Dice Similarity Coefficient for two binary masks. """
    mask1 = mask1.astype(bool)
    mask2 = mask2.astype(bool)
    intersection = np.sum(mask1 & mask2)
    size1 = np.sum(mask1)
    size2 = np.sum(mask2)
    if size1 + size2 == 0:
        return 1.0  # Both masks are empty, return perfect similarity.
    return 2.0 * intersection / (size1 + size2)

def jaccard_index(mask1, mask2):
    """ Compute the Jaccard Index for two binary masks. """
    mask1 = mask1.astype(bool).flatten()
    mask2 = mask2.astype(bool).flatten()
    return jaccard_score(mask1, mask2)

def cohen_kappa(mask1, mask2):
    """ Compute Cohen's Kappa for two binary masks. """
    mask1 = mask1.astype(bool).flatten()
    mask2 = mask2.astype(bool).flatten()
    return cohen_kappa_score(mask1, mask2)

def hausdorff_dist(mask1, mask2):
    """ Compute the Hausdorff Distance for two binary masks. """
    return hausdorff_distance(mask1, mask2)

def volume(mask):
    """ Compute the volume (number of voxels) for a binary mask. """
    return int(np.sum(mask))

def process_subjects(subjects_dir, file_ending1 = '_ChP_mask_T1_mdc.nii', file_ending2 = '_ChP_mask_T1xFLAIR.nii'):
    """ Process NIfTI files for all subjects and compute statistical measures for each pair of masks. """
    subjects = [f.name for f in os.scandir(subjects_dir) if f.is_dir()]

    dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1c, volumes_flair = [], [], [], [], [], []

    for subject in subjects:
        t1mc_path = os.path.join(subjects_dir, subject, subject + file_ending1)
        t1xflair_path = os.path.join(subjects_dir, subject, subject + file_ending2)
        
        if not os.path.exists(t1mc_path) or not os.path.exists(t1xflair_path):
            print(f"Skipping subject {subject}: Missing data")
            continue
        
        t1mc_mask = load_nifti_file(t1mc_path)
        t1xflair_mask = load_nifti_file(t1xflair_path)
        
        dsc = dice_coefficient(t1mc_mask, t1xflair_mask)
        jaccard = jaccard_index(t1mc_mask, t1xflair_mask)
        kappa = cohen_kappa(t1mc_mask, t1xflair_mask)
        hausdorff = hausdorff_dist(t1mc_mask, t1xflair_mask)
        vol_t1mc = volume(t1mc_mask)
        vol_t1xflair = volume(t1xflair_mask)
        
        dsc_scores.append(dsc)
        jaccard_scores.append(jaccard)
        kappa_scores.append(kappa)
        hausdorff_scores.append(hausdorff)
        volumes_t1c.append(vol_t1mc)
        volumes_flair.append(vol_t1xflair)
        
        print(f"Subject {subject}: DSC = {dsc:.4f}, Jaccard = {jaccard:.4f}, Kappa = {kappa:.4f}, Hausdorff = {hausdorff:.4f}, Volume T1c = {vol_t1mc}, Volume FLAIR = {vol_t1xflair}")

    return dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1c, volumes_flair

# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxuser/user/data/T1mc_vs_T1xFLAIR'
subjects_dir ="/data1/user/T1mc_vs_T1xFLAIR_controlled_OK"

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

## _ChP_mask_T1_mdc vx _ChP_mask_T1xFLAIR

In [ ]:
# Define the path to the directory containing the subject data
subjects_dir ="/data1/user/T1mc_vs_T1xFLAIR_controlled_OK" 

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

## _ChP_mask_T1_mdc vs _ChP_mask_T1

In [ ]:
# Define the path to the directory containing the subject data
subjects_dir ="/data1/user/T1mc_vs_T1xFLAIR_controlled_OK" 

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir, file_ending1='_ChP_mask_T1_mdc.nii', file_ending2='_ChP_mask_T1.nii')

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

## _ChP_mask_T1_mdc

In [ ]:
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxuser/user/data/pazienti'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir, file_ending1 = '_ChP_mask_T1_manual_seg.nii', file_ending2 = '_ChP_mask_T1xFLAIR_manual_seg.nii')


In [11]:
# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")



Wilcoxon signed-rank test results:
Volumes: statistic=311.0, p-value=0.05049099592474704

Paired t-test result:
Volumes T1c vs FLAIR: statistic=1.1235842791416, p-value=0.26757080476773304


In [ ]:
def perform_ttest(path):
    t1cm_volumes = []
    t1xflair_volumes = []

    # List all directories in the given path
    subject_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

    for id in subject_dirs:
        t1cm_file = os.path.join(path, id, f"{id}_t1cm.nii")
        t1xflair_file = os.path.join(path, id, f"{id}_t1xflair.nii")

        if os.path.exists(t1cm_file) and os.path.exists(t1xflair_file):
            t1cm_volumes.append(calculate_volume(t1cm_file))
            t1xflair_volumes.append(calculate_volume(t1xflair_file))

    ttest = stats.ttest_ind(t1cm_volumes, t1xflair_volumes)
    return ttest

In [ ]:
# Function to perform t-test on the volume of T1Cm and T1xFLAIR

def ttest_volume(t1cm_volume, t1xflair_volume):
    
    t1cm_volume = np.array(t1cm_volume)
    t1xflair_volume = np.array(t1xflair_volume)
    ttest = stats.ttest_ind(t1cm_volume, t1xflair_volume)
    return ttest

In [ ]:
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxuser/user/data/T1mc_vs_T1xFLAIR_controlled_OK'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir, file_ending1='_ChP_mask_T1_mdc.nii', file_ending2='_ChP_mask_T1.nii')

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs T1: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

In [ ]:
import pandas as pd
# Create a dictionary with the results
results_t1ce_vs_t1 = {
    'DSC': dsc_scores,
    'Jaccard': jaccard_scores,
    'Kappa': kappa_scores,
    'Hausdorff': hausdorff_scores,
    'Volume_T1c': volumes_t1mc,
    'Volume_T1xFLAIR': volumes_t1xflair
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(results_t1ce_vs_t1)

# Save the DataFrame to a CSV file
df.to_csv('/home/linuxuser/user/project_dir/_experiments/00_preanalysis/results_t1ce_vs_t1.csv', index=False)

In [ ]:
# Define the path to the directory containing the subject data
print("T1mc vs T1xFLAIR")
subjects_dir = '/home/linuxuser/user/data/T1mc_vs_T1xFLAIR_controlled_OK'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs T1xFLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

In [ ]:
# save the results in a csv file
import pandas as pd

# Create a dictionary with the results
results = {
    'DSC': dsc_scores,
    'Jaccard': jaccard_scores,
    'Kappa': kappa_scores,
    'Hausdorff': hausdorff_scores,
    'Volume_T1c': volumes_t1mc,
    'Volume_T1xFLAIR': volumes_t1xflair
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('/home/linuxuser/user/project_dir/_experiments/00_preanalysis/results_t1ce_vs_t1xflair.csv', index=False)